In [ ]:
import cv2
import numpy as np
import yaml

class ComputerVision:
    def __init__(self, calibration_file='calibration.yaml'):
        """
        Initializes the ComputerVision object, loads calibration data, and sets up camera capture.
        """
        # Load and use calibration done previously
        with open(calibration_file, 'r') as f:
            calibration_data = yaml.safe_load(f)
        self.mtx = np.array(calibration_data['camera_matrix'])
        self.dist = np.array(calibration_data['dist_coeff'])
        
        # Define ARUCO dictionary to detect
        self.aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
        self.parameters = cv2.aruco.DetectorParameters()

        # Start the camera
        self.cap = cv2.VideoCapture(0)
        ret, img = self.cap.read()
        if not ret:
            print("Error: Could not read from the camera.")
            exit()
        h, w = img.shape[:2]
        self.newcameramtx, self.roi = cv2.getOptimalNewCameraMatrix(self.mtx, self.dist, (w, h), 1, (w, h))

    def get_frame(self):
        """
        Get each frame for observation used in filtering. 
        """
        ret, img = self.cap.read()
        if not ret:
            print("Error: Could not read from the camera.")
        return ret, img

    def process_frame(self):
        """
        Processes each frame to detect ArUco markers, obstacles and start and end points to generate a grid_map for path finding. 
        """
        ret, frame = self.cap.read()
        if not ret:
            print("Error: Could not read frame.")
            return None

        # Frame -> grey -> aruco detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, rejected = cv2.aruco.detectMarkers(gray, self.aruco_dict, parameters=self.parameters)

        start_point = None
        end_point = None

        if ids is not None:
            # Detect start (6) and end (5) markers
            for i, marker_id in enumerate(ids.flatten()):
                marker_center = np.mean(corners[i][0], axis=0)

                if marker_id == 6:  # START
                    start_point = marker_center
                    cv2.circle(frame, tuple(start_point.astype(int)), 5, (0, 255, 0), -1)  # green circle = START

                elif marker_id == 5:  # END
                    end_point = marker_center
                    cv2.circle(frame, tuple(end_point.astype(int)), 5, (0, 0, 255), -1)  # Draw red circle = END

            #  all corner markers are detected (map detected)
            required_ids = {1, 2, 3, 4}
            marker_positions = {marker_id: corners[i][0] for i, marker_id in enumerate(ids.flatten())}
            if required_ids.issubset(marker_positions.keys()):
                rect_corners = np.array([
                    marker_positions[1][0],  # Top-left corner
                    marker_positions[2][1],  # Top-right corner
                    marker_positions[3][2],  # Bottom-right corner
                    marker_positions[4][3],  # Bottom-left corner
                ], dtype=np.float32)

                if rect_corners.shape == (4, 2):
                    # (6:8 aspect ratio for the map)
                    width, height = 600, 800
                    dst_corners = np.array([
                        [0, 0],
                        [width - 1, 0],
                        [width - 1, height - 1],
                        [0, height - 1]
                    ], dtype=np.float32)

                    # Warping & Grey conversion & Binary treshold
                    matrix = cv2.getPerspectiveTransform(rect_corners, dst_corners)
                    warped_image = cv2.warpPerspective(frame, matrix, (width, height))
                    gray_warped = cv2.cvtColor(warped_image, cv2.COLOR_BGR2GRAY)
                    _, binary_map = cv2.threshold(gray_warped, 100, 255, cv2.THRESH_BINARY_INV)

                    warping_size = 125 #magic number (calibration)
                    
                    # mask ArUco markers 5 and 6 (start en end markers)
                    for i, marker_id in enumerate(ids.flatten()):
                        if marker_id in {5, 6}:  # Mask START (6) and END (5)
                            marker_center = np.mean(corners[i][0], axis=0)  # Center of the marker
                            warped_center = cv2.perspectiveTransform(
                                np.array([[marker_center]], dtype=np.float32), matrix)[0][0]
                            warped_center_int = tuple(np.clip(warped_center.astype(int),
                                                              0, [binary_map.shape[1] - 1, binary_map.shape[0] - 1]))
                            cv2.circle(binary_map, warped_center_int, warping_size, 0, -1)  # Mask as white

                    #  map for pathfinding
                    grid_map = np.zeros_like(binary_map, dtype=int)
                    grid_map[binary_map == 255] = 1

                    # Define start (3) and end (2) in the grid_map
                    if start_point is not None:
                        start_warped = cv2.perspectiveTransform(np.array([[start_point]], dtype=np.float32), matrix)[0][0]
                        start_coords = tuple(start_warped.astype(int))
                        grid_map[start_coords[1], start_coords[0]] = 3  # Start

                    if end_point is not None:
                        end_warped = cv2.perspectiveTransform(np.array([[end_point]], dtype=np.float32), matrix)[0][0]
                        end_coords = tuple(end_warped.astype(int))
                        grid_map[end_coords[1], end_coords[0]] = 2  # End

                    return grid_map, rect_corners

        return None

    def get_map(self):
        """
        Captures the grid map from camera and returns it.
        """
        grid_map = None
        while True:
            grid_map, rect_corners = self.process_frame()
            if grid_map is not None:
                print("Grid map generated.")
                break

            # Exit on 'q'
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

        self.cap.release()
        cv2.destroyAllWindows()
        return grid_map, rect_corners


# Usage
#from Computer_Vision import *
cv = ComputerVision()
map, rect_corners = cv.get_map() #for Optimal path

